#### The goal of this notebook is provide an evaluation overview.

In [1]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
import numpy as np
pio.templates.default = "seaborn"

In [2]:
results = pd.read_excel("results.xlsx")
results["Topic"] = results["Query"].copy()
results.loc[1::2, "Topic"] = np.nan
results = results.rename(columns={"Unnamed: 0": "Index"})
plt_df = results.ffill()
plt_df = plt_df[plt_df["Recall"] != 0]


In [3]:
fig = px.histogram(plt_df, x="Topic", y=["Recall", "Cosine Precision",
                                         "Cosine F2", "Cosine F2",
                                         "Cluster Precision", "Cluster F2",
                                         "MVEE Precision", "MVEE F2",
                                         "Hull Precision", "Hull F2"],
                    title="Metrics for Predicted and Baseline Queries", barmode="group",
                   facet_row="Index", facet_row_spacing=0.1)

fig.for_each_trace(lambda t: t.update(x=t.x[::-1], y=t.y[::-1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(yaxis1_title="", yaxis2_title="", yaxis1_dtick=0.2, yaxis2_dtick=0.2)
fig.show()

In [4]:
diff_df = (results[results["Index"] == "Predicted"][["Recall",
                                                    "Cosine Precision", "Cosine F2",
                                                    "Cluster Precision", "Cluster F2",
                                                    "MVEE Precision", "MVEE F2",
                                                    "Hull Precision", "Hull F2"]].values\
      - results[results["Index"] == "Baseline"][["Recall",
                                                "Cosine Precision","Cosine F2", 
                                                "Cluster Precision", "Cluster F2",
                                                "MVEE Precision", "MVEE F2",
                                                "Hull Precision", "Hull F2"]].values).astype(float)

diff_df = pd.DataFrame(np.round(diff_df,3), columns=["Recall",
                                                    "Cosine Precision","Cosine F2", 
                                                    "Cluster Precision", "Cluster F2",
                                                    "MVEE Precision", "MVEE F2",
                                                    "Hull Precision", "Hull F2"], index=results.dropna()["Topic"])
diff_df = diff_df.style.map(lambda x: 'background-color: #6b0801' if x < -0.5 else 'background-color: #a82b22' if x < 0 else '')
diff_df.format("{:.3f}").set_caption("Difference in metrics between predicted and baseline (Negative means the baseline is better)")
display(diff_df.hide(subset=diff_df.index[-7:]))

,Recall,Cosine Precision,Cosine F2,Cluster Precision,Cluster F2,MVEE Precision,MVEE F2,Hull Precision,Hull F2
Topic,,,,,,,,,
Robotic Arthroplasty,0.000,-0.745,-0.550,-0.528,-0.490,-0.741,-0.550,-0.657,-0.550
Soft Robotics,0.083,-0.113,-0.130,-0.147,-0.060,-0.227,-0.220,-0.095,-0.150
Crop Yield Prediction,0.108,-0.262,-0.170,-0.118,-0.010,-0.251,-0.350,-0.216,-0.230
Synthetic Biology,0.310,-0.042,0.030,-0.185,0.140,0.300,0.150,0.294,0.290
Resilience in Business and management,0.148,-0.017,0.040,-0.838,0.200,0.133,0.420,0.072,0.280
Cervical Myelopathy,0.085,-0.288,-0.160,-0.299,-0.240,-0.084,-0.220,-0.026,-0.150
Drones in Agriculture,0.440,-0.158,0.220,0.298,0.370,0.256,0.580,0.154,0.460
Tourism Growth Nexus,0.000,-0.505,-0.070,0.310,0.000,0.000,0.000,0.000,0.000
Sustainable Biofuel Economy,0.260,-0.106,0.240,0.733,0.300,0.730,0.630,0.512,0.700


In [5]:
diff_df.hide(subset=diff_df.index[:15])

,Recall,Cosine Precision,Cosine F2,Cluster Precision,Cluster F2,MVEE Precision,MVEE F2,Hull Precision,Hull F2
Topic,,,,,,,,,
Software Process Line,-0.209,0.270,0.260,0.319,0.140,0.119,0.200,0.162,0.290
Data Stream Processing Latency,-0.073,-0.021,-0.080,-0.321,-0.100,-0.136,-0.130,-0.045,-0.060
Business Process Meta Models,0.269,0.032,0.170,-0.082,0.090,0.499,0.530,0.334,0.520
Multicore Performance Prediction,0.000,0.220,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Cloud Migration,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Software Fault Prediction Metrics,0.542,-0.543,-0.010,0.061,0.440,-0.141,-0.190,-0.048,-0.100
Software Defect Prediction,-0.111,-0.552,-0.270,-0.446,-0.210,-0.564,-0.770,-0.434,-0.750
